In [10]:
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
import os
from dotenv import load_dotenv

In [11]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [12]:
model = ChatGroq(
    api_key=GROQ_API_KEY,
    model="openai/gpt-oss-120b",  # Using the correct model name
    temperature=0.7
)

In [13]:
# create a State
class LLMState(TypedDict):

    question:str
    answer:str

In [14]:
def llm_qa(state:LLMState)->LLMState:

    # extract question from state
    question = state['question']

    # form a prompt
    prompt = f"Answer the following question {question}"

    # ask question to the LLM
    answer = model.invoke(prompt).content

    # update the answer in the state
    state['answer'] = answer

    return state


In [15]:
# create a graph

graph = StateGraph(LLMState)

# add node
graph.add_node('llm_qa',llm_qa)

# add edges
graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa',END)

# compile
workflow = graph.compile()

In [16]:
# Execute
 
initial_state = {
    'question': 'How far the Moon is from the Earth?',
    'answer': ''  # Initialize with empty string
}

final_state = workflow.invoke(initial_state)

print(final_state)

{'question': 'How far the Moon is from the Earth?', 'answer': 'The Moon’s average distance from Earth is about **384\u202f400\u202fkilometers** (≈\u202f238\u202f855\u202fmiles).  \n\n- **Perigee** (closest point): ~\u202f363\u202f300\u202fkm (≈\u202f225\u202f700\u202fmi)  \n- **Apogee** (farthest point): ~\u202f405\u202f500\u202fkm (≈\u202f251\u202f900\u202fmi)  \n\nThese values are averages; the actual distance varies slightly each month as the Moon follows an elliptical orbit.'}


In [17]:
print(final_state['answer'])

The Moon’s average distance from Earth is about **384 400 kilometers** (≈ 238 855 miles).  

- **Perigee** (closest point): ~ 363 300 km (≈ 225 700 mi)  
- **Apogee** (farthest point): ~ 405 500 km (≈ 251 900 mi)  

These values are averages; the actual distance varies slightly each month as the Moon follows an elliptical orbit.
